In [1]:
from train import *

if __name__ == '__main__':
    # hyperparameters
    arg_dict = {
        "env": "11_vs_11_kaggle",
        "num_processes": 1,
        "batch_size": 4,   
        "buffer_size": 2,
        "rollout_len": 30,
        "lstm_size" : 196,
        "k_epoch" : 3,
        "summary_game_window" : 1,
        "model_save_interval" : 100,
        "learning_rate" : 0.0001,
        "gamma" : 0.993,
        "lmbda" : 0.96,
        "entropy_coef" : 0.00003,
#         "trained_model_path" : "logs/[10-28]03.56.37/model_800640.pt",   # default : None
        "trained_model_path" : None,
        "print_mode" : False,
        "latest_ratio" : 0.5,
        
        "encoder" : "encoder_raw",
        "rewarder" : "rewarder_se",
        "model" : "ppo_conv1d"
    }
    

main(arg_dict)

Learner process started
Actor process 0 started
Current Opponent model Num:0, Path:logs/[11-01]23.53.08/model_0.tar successfully loaded
step : 24 loss -0.07454153181364138 data_q 1 summary_q 0
step : 48 loss 0.03549546127518018 data_q 1 summary_q 0
step : 72 loss 0.01939342139909665 data_q 1 summary_q 0
step : 96 loss 0.03522882814286277 data_q 1 summary_q 0
step : 120 loss 0.00950449297670275 data_q 1 summary_q 0
Model saved : logs/[11-01]23.53.08/model_120.tar
step : 144 loss -0.18113083702822527 data_q 1 summary_q 0
step : 168 loss -0.0036020564924304685 data_q 1 summary_q 0
step : 192 loss -0.04629992072780927 data_q 1 summary_q 0
step : 216 loss -0.023730694762586307 data_q 1 summary_q 0
step : 240 loss 0.03916484800477823 data_q 1 summary_q 0
Model saved : logs/[11-01]23.53.08/model_240.tar
step : 264 loss 0.036621264492472015 data_q 1 summary_q 0
step : 288 loss 0.03481739363633096 data_q 1 summary_q 0
score 1.0 total reward 5.000000000000005
Current Opponent model Num:2, Path:l

Process Process-1:
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/dist-packages/tensorboardX/event_file_writer.py", line 202, in run
    data = self._queue.get(True, queue_wait_duration)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 108, in get
    res = self._recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 383, in _recv
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()

  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self

KeyboardInterrupt: 

In [1]:
import random

x = random.random()
print(x)



0.15057106765923955


In [1]:
x = [1,4,2]
print(x[3:],x[:3])

[] [1, 4, 2]
